## NullFraud Predictive Modeling

## Loaded the required library

In [1]:
library(tidymodels)
library(tidyverse)
library(readxl)
library(dplyr)

# Setting a seed so our analysis is random but reproducible
set.seed(2005)

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ recipes      1.0.8
✔ dials        1.2.0     ✔ rsample      1.2.0
✔ dplyr        1.1.3     ✔ tibble       3.2.1
✔ ggplot2      3.5.0     ✔ tidyr        1.3.0
✔ infer        1.0.5     ✔ tune         1.1.2
✔ modeldata    1.2.0     ✔ workflows    1.1.3
✔ parsnip      1.1.1     ✔ workflowsets 1.0.1
✔ purrr        1.0.2     ✔ yardstick    1.2.0

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Learn how to get started at https://www.tidymodels.org/start/

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ lubridate 1.9.2     ✔ stringr   1.5.0
── Conflicts

## Cleaning and wrangling the data 

In [2]:
# Read the data
fraud_data <- read_excel("BOLT Data Set.xlsx")

#Rename the column

fraud_data <- fraud_data |>
rename("Card_Identifier" = "Card Identifier", "Transaction_Date" = "Transaction Date", "Transaction_Time" = "Transaction Time",
       "Risk_Assessment" = "Risk Assessment", "Payment_Method" = "Payment Method", "Transaction_Value" = "Transaction Value",
       "Merchant_Location" = "Merchant Location", "Card_Present_Status" = "Card Present Status", 
       "Chip_Usage" = "Chip Usage" , "Cross_border_Transaction" = "Cross-border Transaction (Yes/No)",
        "Acquiring_Institution_ID" = "Acquiring Institution ID", "Merchant_Identifier" = "Merchant Identifier", 
       "Merchant_Category_Code" = "Merchant Category Code (MCC)", "Fraud_Indicator" = "Fraud Indicator (Yes/No)") 


In [3]:
# Converting to correct data types
numeric_column_names <- c("Risk_Assessment", "Transaction_Value")
factor_column_names <- c("Card_Identifier", "Payment_Method", "Merchant_Location", "Card_Present_Status", "Chip_Usage",
                       "Cross_border_Transaction", "Acquiring_Institution_ID", "Merchant_Identifier", 
                        "Fraud_Indicator")
fraud_data <- fraud_data |>
    mutate(across(all_of(numeric_column_names), as.numeric)) |>
    mutate(across(all_of(factor_column_names), as_factor)) 

fraud_data$Chip_Usage <- ifelse(fraud_data$Chip_Usage == "Yes", 1, 0)
fraud_data$Cross_border_Transaction <- ifelse(fraud_data$Cross_border_Transaction == "Yes", 1, 0)
fraud_data$Fraud_Indicator <- ifelse(fraud_data$Fraud_Indicator == "Yes", 1, 0)
fraud_data$Card_Present_Status <- ifelse(fraud_data$Card_Present_Status == "Yes", 1, 0)
    
glimpse(fraud_data)

Rows: 100,000
Columns: 14
$ Card_Identifier          <fct> card 1, card 2, card 3, card 4, card 5, card …
$ Transaction_Date         <dttm> 2023-05-11, 2023-06-05, 2023-06-05, 2023-07-…
$ Transaction_Time         <chr> "16:22:14.0", "15:16:35.0", "11:57:40.0", "18…
$ Risk_Assessment          <dbl> 362, 602, 482, 947, 1382, 612, 1327, 1037, 49…
$ Payment_Method           <fct> Paypass - Contactless, Online, Unknown, Onlin…
$ Transaction_Value        <dbl> 13.98, 24.64, 15.00, 30.56, 50.85, 75.77, 0.0…
$ Merchant_Location        <fct> USA, USA, USA, USA, USA, USA, USA, USA, USA, …
$ Card_Present_Status      <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ Chip_Usage               <dbl> 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, …
$ Cross_border_Transaction <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, …
$ Acquiring_Institution_ID <fct> acquirer 1, acquirer 2, acquirer 3, acquirer …
$ Merchant_Identifier      <fct> merchant 1, merchant 2, merchant 3, merchant …
$ Merchant_Cat

## Splitting the data 

In [4]:
#Split the data into training and testing set
fraud_data_split <- fraud_data |>
    initial_split(prop = 0.75, strata = "Fraud_Indicator")

fraud_training <- training(fraud_data_split)
fraud_testing <- testing(fraud_data_split)


## Stepwise Selection

In [5]:
#Loaded the required library
#install.packages('MASS')
library(MASS)
library(caret)


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: lattice


Attaching package: ‘caret’


The following objects are masked from ‘package:yardstick’:

    precision, recall, sensitivity, specificity


The following object is masked from ‘package:purrr’:

    lift




In [6]:
logit_1 <- glm(Fraud_Indicator ~ Risk_Assessment + Transaction_Value
               + Card_Present_Status + Chip_Usage + Cross_border_Transaction, 
               family = binomial,data = fraud_training )

In [7]:
logit_2 <- stepAIC(logit_1)

Start:  AIC=2397.1
Fraud_Indicator ~ Risk_Assessment + Transaction_Value + Card_Present_Status + 
    Chip_Usage + Cross_border_Transaction


Step:  AIC=2397.1
Fraud_Indicator ~ Risk_Assessment + Transaction_Value + Chip_Usage + 
    Cross_border_Transaction

                           Df Deviance    AIC
- Transaction_Value         1   2388.1 2396.1
- Cross_border_Transaction  1   2388.1 2396.1
<none>                          2387.1 2397.1
- Chip_Usage                1   2406.8 2414.8
- Risk_Assessment           1   2696.5 2704.5

Step:  AIC=2396.1
Fraud_Indicator ~ Risk_Assessment + Chip_Usage + Cross_border_Transaction

                           Df Deviance    AIC
- Cross_border_Transaction  1   2389.2 2395.2
<none>                          2388.1 2396.1
- Chip_Usage                1   2407.8 2413.8
- Risk_Assessment           1   2697.0 2703.0

Step:  AIC=2395.15
Fraud_Indicator ~ Risk_Assessment + Chip_Usage

                  Df Deviance    AIC
<none>                 2389.2 2395.